# Prediction

## importation des données

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
pricing_requests = pd.read_csv("price_dataset1.csv",index_col =0)
pricing_requests = pricing_requests.join(hotels, on=['hotel_id', 'city'])
pricing_requests


In [ ]:
pricing_requests

In [ ]:
# def split_dataset(dataset, test_ratio=0.30):
#   """Splits a panda dataframe in two."""
#   test_indices = np.random.rand(len(dataset)) < test_ratio
#   return dataset[~test_indices], dataset[test_indices]


In [ ]:
# train_ds_pd, test_ds_pd = split_dataset(pricing_requests)

In [ ]:
pip install tensorflow_decision_forests --upgrade

In [2]:
import tensorflow_decision_forests

  Referenced from: /Users/user/opt/anaconda3/lib/python3.9/site-packages/tensorflow_decision_forests/tensorflow/ops/training/training.so (which was built for Mac OS X 12.3)
  Expected in: /usr/lib/libc++.1.dylib



NotFoundError: dlopen(/Users/user/opt/anaconda3/lib/python3.9/site-packages/tensorflow_decision_forests/tensorflow/ops/training/training.so, 6): Symbol not found: __ZNKSt3__115basic_stringbufIcNS_11char_traitsIcEENS_9allocatorIcEEE3strEv
  Referenced from: /Users/user/opt/anaconda3/lib/python3.9/site-packages/tensorflow_decision_forests/tensorflow/ops/training/training.so (which was built for Mac OS X 12.3)
  Expected in: /usr/lib/libc++.1.dylib


In [ ]:
label ="price"
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(pricing_requests, label=label, task=tfdf.keras.Task.REGRESSION)
# test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)

## Tuner potentiel

In [ ]:
# # Configure the tuner.

# # Create a Random Search tuner with 50 trials.
# tuner = tfdf.tuner.RandomSearch(num_trials=50)

# # Define the search space.
# #
# # Adding more parameters generaly improve the quality of the model, but make
# # the tuning last longer.

# tuner.choice("min_examples", [2, 5, 7, 10])
# tuner.choice("categorical_algorithm", ["CART", "RANDOM"])

# # Some hyper-parameters are only valid for specific values of other
# # hyper-parameters. For example, the "max_depth" parameter is mostly useful when
# # "growing_strategy=LOCAL" while "max_num_nodes" is better suited when
# # "growing_strategy=BEST_FIRST_GLOBAL".

# local_search_space = tuner.choice("growing_strategy", ["LOCAL"])
# local_search_space.choice("max_depth", [3, 4, 5, 6, 8])

# # merge=True indicates that the parameter (here "growing_strategy") is already
# # defined, and that new values are added to it.
# global_search_space = tuner.choice("growing_strategy", ["BEST_FIRST_GLOBAL"], merge=True)
# global_search_space.choice("max_num_nodes", [16, 32, 64, 128, 256])

# tuner.choice("use_hessian_gain", [True, False])
# tuner.choice("shrinkage", [0.02, 0.05, 0.10, 0.15])
# tuner.choice("num_candidate_attributes_ratio", [0.2, 0.5, 0.9, 1.0])

# # Uncomment some (or all) of the following hyper-parameters to increase the
# # quality of the search. The number of trial should be increased accordingly.

# # tuner.choice("split_axis", ["AXIS_ALIGNED"])
# # oblique_space = tuner.choice("split_axis", ["SPARSE_OBLIQUE"], merge=True)
# # oblique_space.choice("sparse_oblique_normalization",
# #                      ["NONE", "STANDARD_DEVIATION", "MIN_MAX"])
# # oblique_space.choice("sparse_oblique_weights", ["BINARY", "CONTINUOUS"])
# # oblique_space.choice("sparse_oblique_num_projections_exponent", [1.0, 1.5])


## Compilation et prédiction

In [ ]:
model = tfdf.keras.RandomForestModel(task = tfdf.keras.Task.REGRESSION)
model.compile(metrics=["mse"])
model.fit(x=train_ds, verbose =0)


In [ ]:
# evaluation = model.evaluate(test_ds ,return_dict=True)
# si on a split nos données

# print(evaluation)

In [ ]:
test = pd.read_csv("test_set.csv",index_col=0)

test = test.join(hotels, on=['hotel_id', 'city'])

test = test.drop(["order_requests"],axis =1)

test_df = tfdf.keras.pd_dataframe_to_tf_dataset(test)

A =model.predict(test_df)

In [ ]:
test_df

In [ ]:
test

In [ ]:
Submission = pd.DataFrame(A)

Submission.to_csv("submission1.csv",header = ["price"],index_label="index")

# Gradio

In [ ]:
def DfFromInput(city,date,language,mobile,avatar_id,hotel_id,stock):
    d = {'city':[city] ,'date':[date] , 'language':[language] ,'mobile':[mobile] ,'avatar_id':[avatar_id] ,
         'hotel_id':[hotel_id],'stock':[stock]}
    df=pd.DataFrame(data=d)
    return df

def AddData(city,date,language,mobile,avatar_id,hotel_id,stock):
    df = DfFromInput(city,date,language,mobile,avatar_id,hotel_id,stock)
    hotels = pd.read_csv('features_hotels.csv', index_col=['hotel_id', 'city'])
    df = df.join(hotels, on=['hotel_id', 'city'])
    return df

def Price(city,date,language,mobile,avatar_id,hotel_id,stock):
    df =AddData(city,date,language,mobile,avatar_id,hotel_id,stock)
    df =tfdf.keras.pd_dataframe_to_tf_dataset(df)
    price =model.predict(df,verbose =0)
    price=price[0,0]
    return price

TestDF = AddData("vilnius",21,"romanian",0,1,161,46)
price = Price("vilnius",21,"romanian",0,1,161,46)
price

In [ ]:
test

In [ ]:
testGrad = pd.read_csv("price_dataset1.csv",index_col =0)
testGrad

In [ ]:
pip install gradio

In [ ]:
import gradio as gr

In [ ]:
(city,date,language,mobile,avatar_id,hotel_id,stock)

In [ ]:
test = gr.Interface(fn = Price,
                    inputs= [
                        gr.Radio(["amsterdam","copenhagen","madrid","paris","rome",
                                          "sofia","valletta","vienna","vilnius"]),
                        gr.Slider(0,44,step =1,value =0),
                        gr.Radio(["austrian","belgian","bulgarian","croatian","cypriot",
                                       "czech","danish","dutch","estonian","finnish","french",
                                       "german","greek","hungarian","irish","italian","latvian",
                                       "lithuanian","luxembourgish","maltese","polish","portuguese",
                                       "romanian","slovakian","slovene","spanish","swedish"]),
                        gr.Slider(0,1,step=1,value=0),
                        gr.Number(value=111612),
                        gr.Number(),
                        gr.Number()
                        
                    ],
                    
                    outputs="number")
test.launch()